In [1]:
!pip install -q tensorflow-recommenders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
#from surprise import Dataset, NormalPredictor, Reader
#from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
#from surprise import accuracy
#from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
#from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict
import joblib

In [3]:
train_rmse0 = pd.read_csv('rmse/rmse_folds_actual/actual_rmse_train0.csv', header=None)
test_rmse0 = pd.read_csv('rmse/rmse_folds_actual/actual_rmse_test0.csv', header=None)

In [4]:
train_rmse0.head()

,0,1,2
0,1590329,7627,3.0
1,576570,14606,4.0
2,1757599,9156,3.0
3,1048852,15107,4.0
4,2187348,14644,3.0


In [5]:
test_rmse0.head()

,0,1,2
0,573762,9438,4.0
1,1560019,5760,4.0
2,2120159,1542,1.0
3,1388182,12034,1.0
4,142077,15755,3.0


In [6]:
train_rmse0.columns = ['user_id', 'movie_id', 'rating']
test_rmse0.columns = ['user_id', 'movie_id', 'rating']

In [7]:
train_movie_ids = list(df_rating['movie_id'].unique())
train_user_ids = list(df_rating['user_id'].unique())

In [8]:
class DeepRecRmseModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        embedding_dimension = 16
        
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_user_ids)+1, embedding_dimension)
        ])
        
        self.movie_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_movie_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_movie_ids)+1, embedding_dimension)
        ])
        
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
    def call(self, inputs):
        user_id, movie_id = inputs
        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_id)
        return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))
    
    

In [9]:
class NetflixRmseModel(tfrs.models.Model):
    
    def __init__(self):
        super().__init__()
        self.rmse_model: tf.keras.Model = DeepRecRmseModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(),
                                                             metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.rmse_model((features['user_id'], features['movie_id']))
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        labels = features.pop('rating')
        predicted_ratings = self(features)
        return self.task(labels=labels, predictions=predicted_ratings)

In [10]:
for i in range(5):
    train_file1 = f'rmse/rmse_folds_actual/actual_rmse_train{i}.csv'
    test_file1 = f'rmse/rmse_folds_actual/actual_rmse_train{i}.csv'
    df_train1 = pd.read_csv(train_file1, header=None)
    df_test1 = pd.read_csv(test_file1, header=None)
    df_train1.columns = ['user_id', 'movie_id', 'rating']
    df_test1.columns = ['user_id', 'movie_id', 'rating']
    
    deep_model1 = NetflixRmseModel()
    deep_model1.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
    
    deep_model2 = NetflixRmseModel()
    deep_model2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
    
    df_train_tf1 = tf.data.Dataset.from_tensor_slices(dict(df_train1))
    df_test_tf1 = tf.data.Dataset.from_tensor_slices(dict(df_test1))
    
    df_train_tf1 = df_train_tf1.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']})
    
    df_test_tf1 = df_test_tf1.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']})
    
    df_train_tf1 = df_train_tf1.batch(8192).cache()
    df_test_tf1 = df_test_tf1.batch(8192).cache()
    
    history_baseline1 = deep_model1.fit(df_train_tf1, epochs=10)
    joblib.dump(deep_model1, f'model_rmse{i}a.pkl')
    
    history_baseline2 = deep_model2.fit(df_test_tf1, epochs=10)
    joblib.dump(deep_model2, f'model_rmse{i}b.pkl')
    

Epoch 1/10
301/301 [==============================] - 89s 275ms/step - root_mean_squared_error: 1.1208 - loss: 1.2554 - regularization_loss: 0.0000e+00 - total_loss: 1.2554
Epoch 2/10
301/301 [==============================] - 74s 246ms/step - root_mean_squared_error: 1.0432 - loss: 1.0881 - regularization_loss: 0.0000e+00 - total_loss: 1.0881
Epoch 3/10
301/301 [==============================] - 74s 246ms/step - root_mean_squared_error: 1.0384 - loss: 1.0781 - regularization_loss: 0.0000e+00 - total_loss: 1.0781
Epoch 4/10
301/301 [==============================] - 74s 247ms/step - root_mean_squared_error: 1.0358 - loss: 1.0727 - regularization_loss: 0.0000e+00 - total_loss: 1.0727
Epoch 5/10
301/301 [==============================] - 75s 248ms/step - root_mean_squared_error: 1.0340 - loss: 1.0690 - regularization_loss: 0.0000e+00 - total_loss: 1.0690
Epoch 6/10
301/301 [==============================] - 74s 246ms/step - root_mean_squared_error: 1.0326 - loss: 1.0661 - regularization_

INFO:tensorflow:Assets written to: ram://e6b447f6-1158-4d92-8097-c993575ae58b/assets


INFO:tensorflow:Assets written to: ram://e6b447f6-1158-4d92-8097-c993575ae58b/assets


Epoch 1/10
301/301 [==============================] - 88s 275ms/step - root_mean_squared_error: 1.1174 - loss: 1.2480 - regularization_loss: 0.0000e+00 - total_loss: 1.2480
Epoch 2/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0440 - loss: 1.0898 - regularization_loss: 0.0000e+00 - total_loss: 1.0898
Epoch 3/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0388 - loss: 1.0790 - regularization_loss: 0.0000e+00 - total_loss: 1.0790
Epoch 4/10
301/301 [==============================] - 74s 244ms/step - root_mean_squared_error: 1.0361 - loss: 1.0733 - regularization_loss: 0.0000e+00 - total_loss: 1.0733
Epoch 5/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0342 - loss: 1.0695 - regularization_loss: 0.0000e+00 - total_loss: 1.0695
Epoch 6/10
301/301 [==============================] - 74s 248ms/step - root_mean_squared_error: 1.0328 - loss: 1.0666 - regularization_

INFO:tensorflow:Assets written to: ram://4ea7ae18-1144-49ed-ba10-59b5c0368286/assets


...

INFO:tensorflow:Assets written to: ram://e0483399-860f-40db-858b-f6c9ccfa6573/assets


INFO:tensorflow:Assets written to: ram://e0483399-860f-40db-858b-f6c9ccfa6573/assets


Epoch 1/10
301/301 [==============================] - 88s 275ms/step - root_mean_squared_error: 1.1172 - loss: 1.2476 - regularization_loss: 0.0000e+00 - total_loss: 1.2476
Epoch 2/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0445 - loss: 1.0909 - regularization_loss: 0.0000e+00 - total_loss: 1.0909
Epoch 3/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0393 - loss: 1.0801 - regularization_loss: 0.0000e+00 - total_loss: 1.0801
Epoch 4/10
301/301 [==============================] - 73s 244ms/step - root_mean_squared_error: 1.0365 - loss: 1.0744 - regularization_loss: 0.0000e+00 - total_loss: 1.0744
Epoch 5/10
301/301 [==============================] - 74s 246ms/step - root_mean_squared_error: 1.0347 - loss: 1.0706 - regularization_loss: 0.0000e+00 - total_loss: 1.0706
Epoch 6/10
301/301 [==============================] - 74s 245ms/step - root_mean_squared_error: 1.0334 - loss: 1.0679 - regularization_

INFO:tensorflow:Assets written to: ram://eed474f1-1343-48ef-ab00-eb80b2e2ca11/assets


INFO:tensorflow:Assets written to: ram://eed474f1-1343-48ef-ab00-eb80b2e2ca11/assets
